<a href="https://colab.research.google.com/github/cmiglio/signal_processing_with_python/blob/master/signal_processing_with_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Signal analysis with python

*Carolina Migliorelli*


The purpose of this workshop is to show an example on how to work with Python with signals. 
We will give some general ideas on how to **read, process and visualize data**

We will use some common python toolboxes: 
* [numpy](https://numpy.org/): It contains the main functions to work with matrices (very similar to Matlab)
* [pandas](https://pandas.pydata.org/): A way of organizing the data
* [seaborn](https://seaborn.pydata.org/): A visualization library with nice plots (that use in its backgroud [matplotlib](https://matplotlib.org/).
* [scipy](https://www.scipy.org/): We will use scipy to load mat files.
* [mne](https://mne.tools/stable/index.html): A package for analyzing human neurophysiological data

For this workshop, we will load data from one epileptic patient and we will extract some events of interest. We will plot the time-frequency representation of these events using the stockwell transform and we will save into a dataframe structure (a table) some meaningful features.

## 0. Installing packages
If you don't have installed the toolboxes, then you should install them. 

In [0]:
!pip install numpy pandas seaborn scipy mne
!git clone https://github.com/cmiglio/signal_processing_with_python.git

     |████████████████████████████████| 6.4MB 3.4MB/s 
Cloning into 'signal_processing_with_python'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 0), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


## 1. Importing packages 
We will start importing the packages. When we use *import* we import the whole package, with the name that we put after *as*. If we don't want to import the whole package but only some functions, we can use *from*. 

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.io import loadmat
import mne
from mne.time_frequency import (tfr_array_stockwell) 

## 2. Loading the data
### 2.1. Loading the data using loadmat (from scipy)
We will load the data that, in this case, is stored in a *.mat* file. We can import data with other formats using MNE ([see here](https://mne.tools/stable/auto_tutorials/io/plot_20_reading_eeg_data.html#sphx-glr-auto-tutorials-io-plot-20-reading-eeg-data-py)). But however, .TRC (micromed) files are not currently included in MNE distribution, so what I did is reading the data with matlab and then save it into a .mat file. If you are interested in knowing how to create the .mat file, I cand send you my routines. 

In [0]:
# Files were saved into a structure, you should know the names of each field
# in the structure. If not, you can check de names usign structure.keys().
# The squeeze_me parameter must be set to True.
structure = loadmat('signal_processing_with_python/signals.mat',squeeze_me=True)
data = structure['data']
sfreq = structure['sfreq']
ch_names = structure['ch_names']
ch_types = structure['ch_types']
differential = structure['differential']
average = structure['average']
needle_names = structure['needle_names']
needle = structure['needle1']
pos = structure['pos']

### 2.2. Creating an mne raw array
To work with MNE (and use filters for example), we have to create a data structure that MNE understands.  

In [0]:
# We have to change the numpy arrays to lists
info = mne.create_info(ch_names=ch_names.tolist(), sfreq=sfreq, ch_types=ch_types.tolist())
raw = mne.io.RawArray(data, info)

Creating RawArray with float64 data, n_channels=20, n_times=61440
    Range : 0 ... 61439 =      0.000 ...    59.999 secs
Ready.


## 3. Processing data
### 3.1. Filtering with MNE
We will filter the data above 80 Hz and we will notch the data to remove the line noise (50Hz and the harmonics)

In [0]:
raw.notch_filter([50, 100, 150, 200, 250], fir_design='firwin',verbose=False)
raw80 = raw.copy() # We create a copy of our data
raw80.filter(80,None,fir_design='firwin',verbose=False)


<RawArray  |  None, n_channels x n_times : 20 x 61440 (60.0 sec), ~9.4 MB, data loaded>

### 3.2. Obtaining the envelope of the signal and computing the threshold
We want to obtain the envelope of the signal using hilbert and to obtain the events that are higher than a threshold. We will set the threshold to:
``` python
mean + 3*sd.
```
(Other more sophisticated methods to obtain the threshold could be applyied)


We will compute this only for one channel, but I leave you the code to compute this for all the channels (using for in python is easy, but a little bit different than in Matlab)


The code to iterate trough channels is: 
```python

# We obtain a vector that have the size of the channels and other vector with 
# the same size for the threshold
channels_vector = np.arange(0,len(raw80.ch_names)) 
th = np.zeros(channels_vector.shape) 

for chan_n in channels_vector: # We iterate trough the channels
    data_80 = raw80.get_data(picks=chan_n) # We obtain the data for one channel
    data_hil = np.abs(raw_hilb.get_data(picks=chan_n)) # We obtain the envelope for one channel
    th[chan_n] = np.mean(data_hil) + 3*np.std(data_hil)
    
    # All this code is to detect events of interest that will be saved into eoi
    # Where function is similar to find
    change = np.diff((data_hil[0,:]>th[chan_n])*1)
    if change[np.where(change)[0][0]]==-1:
        change[np.where(change)[0][0]]=0
    ch_pos=np.sum(change==1)
    ch_neg=np.sum(change==-1)
    if ch_pos>ch_neg:
        # Si hay 1 positivo mas que negativo, entonces es que hemos acabado encima del threshold--> descartamos el ultimo positivo
        p_aux_ini=np.where(change==1)[0][:-2]
        p_aux_fin=np.where(change==-1)[0]
    elif ch_pos<ch_neg:
        # Si hay 1 negativo más que un positivo, entonces es que he empezado encima del threshold--> descartamos el primer negativo
        p_aux_ini=np.where(change==1)[0]
        p_aux_fin=np.where(change==-1)[0][1:]
    elif ch_pos==ch_neg:
        p_aux_ini=np.where(change==1)[0]
        p_aux_fin=np.where(change==-1)[0]
    eoi = np.array(list(zip(p_aux_ini,p_aux_fin)))
```

In [0]:
raw_hilb = raw80.copy() # We create a copy of our filtered data
raw_hilb.apply_hilbert() # We apply hilbert to obtain the envelope

chan_n = 0 # We will compute this for the first channel (in python the first index is 0)

data_80 = raw80.get_data(picks=chan_n)
data_hil = np.abs(raw_hilb.get_data(picks=chan_n))
th = np.mean(data_hil) + 3*np.std(data_hil)
change = np.diff((data_hil[0,:]>th)*1)
if change[np.where(change)[0][0]]==-1:
    change[np.where(change)[0][0]]=0
ch_pos=np.sum(change==1)
ch_neg=np.sum(change==-1)
if ch_pos>ch_neg:
            # Si hay 1 positivo mas que negativo, entonces es que hemos acabado encima del threshold--> descartamos el ultimo positivo
    p_aux_ini=np.where(change==1)[0][:-2]
    p_aux_fin=np.where(change==-1)[0]
elif ch_pos<ch_neg:
     # Si hay 1 negativo más que un positivo, entonces es que he empezado encima del threshold--> descartamos el primer negativo
    p_aux_ini=np.where(change==1)[0]
    p_aux_fin=np.where(change==-1)[0][1:]
elif ch_pos==ch_neg:
    p_aux_ini=np.where(change==1)[0]
    p_aux_fin=np.where(change==-1)[0]
eoi = np.array(list(zip(p_aux_ini,p_aux_fin)))


Lets see what we have in *eoi*

In [0]:
print(eoi)

[[23813 23867]
 [23872 23887]
 [23888 23893]
 [23895 23899]
 [23917 23942]
 [23943 23987]
 [24051 24101]
 [31403 31404]
 [31405 31409]
 [31410 31427]
 [31475 31500]
 [49991 49995]
 [49998 49999]
 [50000 50031]
 [50033 50037]
 [50092 50096]
 [50099 50105]
 [50106 50128]
 [50129 50130]
 [50136 50138]]
